In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCVsK6xyXxhMI_BmOY1GFxu3uBr3STcoGiOWCJhuECHwknpiW04P1wblb674-uK3gjZlw&usqp=CAU)https://www.bis.org/ifc/events/ifc_9thconf/Petropoulos.pdf

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #001f3f;"><b style="color:#03e8fc;">Boruta all-relevant feature selection method</b></h1></center>


"Boruta is an all relevant feature selection method, while most other are minimal optimal; this means it tries to find all features carrying information usable for prediction, rather than finding a possibly compact subset of features on which some classifier has a minimal error."

"Why bother with all relevant feature selection? When you try to understand the phenomenon that made your data, you should care about all factors that contribute to it, not just the bluntest signs of it in context of your methodology (yes, minimal optimal set of features by definition depends on your classifier choice)."

https://github.com/scikit-learn-contrib/boruta_py

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/6345.csv")
df.head()

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

import datetime

#Split date: year/month/date

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

df[['year','month','date']] = df['time'].apply(lambda x : pd.Series(str(x).split('-')))

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

df.drop('time', inplace=True, axis =1)

In [ ]:
df.head()

In [ ]:
#Code by Parul Pandey  https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python


from sklearn.impute import SimpleImputer
df1 = df.copy()
#setting strategy to 'mean' to impute by the mean
mean_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
df1.iloc[:,:] = mean_imputer.fit_transform(df1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in df1.columns:
    if df1[c].dtype=='float16' or  df1[c].dtype=='float32' or  df1[c].dtype=='float64':
        df1[c].fillna(df[c].mean())

#fill in -999 for categoricals
df1 = df1.fillna(-999)
# Label Encoding
for f in df1.columns:
    if df1[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df1[f].values))
        df1[f] = lbl.transform(list(df1[f].values))
        
print('Labelling done.')

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

# lets create a feature matrix
feature_names =    ['engagement_index','perc_access', 'month', 'date']

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

# define function for creating y
def yfromX(X):
    y = X['engagement_index'] + X['perc_access']**2 + np.sin(3 * X['month']) + (X['date'])
    return y

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

from sklearn.model_selection import train_test_split

# create x and y
np.random.seed(0)

X = pd.DataFrame(np.random.normal(size = (20000, len(feature_names))), columns = feature_names)
y = yfromX(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

# linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

lr_train_preds = lr.predict(X_train)
lr_test_preds = lr.predict(X_test)

lr_train_mae = mean_absolute_error(y_train, lr_train_preds)
lr_test_mae = mean_absolute_error(y_test, lr_test_preds)

lr_fi = PermutationImportance(lr, cv = 'prefit', n_iter = 3).fit(X_train, y_train).feature_importances_

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

# Random Forest 
rf = RandomForestRegressor(max_depth=5)
rf.fit(X_train, y_train)

rf_train_preds = rf.predict(X_train)
rf_test_preds = rf.predict(X_test)

rf_train_mae = mean_absolute_error(y_train, rf_train_preds)
rf_test_mae = mean_absolute_error(y_test, rf_test_preds)

rf_fi = rf.feature_importances_

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

# Light GBM
lgb = LGBMRegressor(max_depth=5)
lgb.fit(X_train, y_train)

lgb_train_preds = lgb.predict(X_train)
lgb_test_preds = lgb.predict(X_test)

lgb_train_mae = mean_absolute_error(y_train, lgb_train_preds)
lgb_test_mae = mean_absolute_error(y_test, lgb_test_preds)

lgb_fi = lgb.feature_importances_

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter 
import matplotlib.ticker as mtick

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

from boruta import BorutaPy

new_rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)

boruta_selector = BorutaPy(new_rf, n_estimators = 'auto', random_state = 0)
boruta_selector.fit(np.array(X_train), np.array(y_train))

boruta_ranking = boruta_selector.ranking_
selected_features = np.array(feature_names)[boruta_ranking <= 2]

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

boruta_ranking = pd.DataFrame(data=boruta_ranking, index=X_train.columns.values, columns=['values'])
boruta_ranking['Variable'] = boruta_ranking.index
boruta_ranking.sort_values(['values'], ascending=True, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter 
import matplotlib.ticker as mtick

In [ ]:
#Code by Ajay Sampath https://www.kaggle.com/ajaysamp/boruta-beats-em-all-new-look-at-feature-selection

fig,ax = plt.subplots(figsize=(8,4))
ax = sns.barplot(x='values',y='Variable',data=boruta_ranking, color='b')
plt.title('Boruta Feature Ranking')
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()

#Boruta's ranking output (Rank 1: confirmed, Rank 2: some influence, Above Rank 3: Rejected

The output from Boruta is a ranking number. So we can divide them into categories are need. All features with a rank 1 mean that these have a confirmed effect on the target variable. A rank 2 indicates that there is some influence. Anything above 3 can be rejected as it does not have any infuence.

Using Boruta, we picked all the correct dependent variables and rejected the noise(??).

Boruta can be a very powerful tool to use and check feature importances from traditional methods.